<a href="https://colab.research.google.com/github/nitingopi/agent_from_scratch/blob/main/AgentPro_Part_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance matplotlib fpdf qiskit python-pptx openai --quiet

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 2.1 MB/s eta 0:00:00


In [ ]:
from openai import OpenAI
import os
from google.colab import userdata
import base64
import requests
from PIL import Image
from io import BytesIO


os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [ ]:

import subprocess
import tempfile
import re
import importlib
import sys

class AgentPro:


    def generate_code(self, prompt):
        client = OpenAI()
        response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
          {"role": "system", "content": "You are a Python code generator. Respond only with executable Python code, no explanations or comments except for required pip installations at the top."},
          {"role": "user", "content": f"Generate Python code to {prompt}. If you need to use any external libraries, include a comment at the top of the code listing the required pip installations."}
        ],
        max_tokens=4000,
        temperature=0.7,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
        )
        code = re.sub(r'^```python\n|^```\n|```$', '', response.choices[0].message.content, flags=re.MULTILINE)
        code_lines = code.split('\n')
        while code_lines and not (code_lines[0].startswith('import') or code_lines[0].startswith('from') or code_lines[0].startswith('#')):
            code_lines.pop(0)

        return '\n'.join(code_lines)

        #return


    def install_libraries(self, code):
        libraries = re.findall(r'#\s*pip install\s+([\w-]+)', code)
        if libraries:
            print("Installing required libraries...")
            for lib in libraries:
                try:
                    importlib.import_module(lib.replace('-', '_'))
                    print(f"{lib} is already installed.")
                except ImportError:
                    print(f"Installing {lib}...")
                    subprocess.check_call([sys.executable, "-m", "pip", "install", lib])
            print("Libraries installed successfully.")


    def execute_code(self, code):
        with tempfile.NamedTemporaryFile(mode='w', suffix='.py', delete=False) as temp_file:
            temp_file.write(code)
            temp_file_path = temp_file.name

        try:
            result = subprocess.run(['python', temp_file_path], capture_output=True, text=True, timeout=30)
            output = result.stdout
            error = result.stderr
        except subprocess.TimeoutExpired:
            output = ""
            error = "Execution timed out after 30 seconds."
        finally:
            os.unlink(temp_file_path)

        return output, error

    def run(self, prompt):
        print(f"Generating code for: {prompt}")
        code = self.generate_code(prompt)
        print("Generated code:")
        print(code)
        print("\nExecuting code...")
        output, error = self.execute_code(code)

        if output:
            print("Output:")
            print(output)
        if error:
            print("Error:")
            print(error)



In [ ]:
# Example usage
if __name__ == "__main__":
    agent = AgentPro()
    agent.run("make a detailed deck on the best way to make a colony on Mars and save it as Mars.pptx")

Generating code for: make a detailed deck on the best way to make a colony on Mars and save it as Mars.pptx
Generated code:
# Required installations:
# pip install python-pptx

from pptx import Presentation
from pptx.util import Inches

# Create a presentation object
prs = Presentation()

# Slide 1: Title Slide
slide_layout = prs.slide_layouts[0]
slide = prs.slides.add_slide(slide_layout)
title = slide.shapes.title
subtitle = slide.placeholders[1]
title.text = "Colonizing Mars: A Step-by-Step Guide"
subtitle.text = "Exploring the Red Planet"

# Slide 2: Introduction
slide_layout = prs.slide_layouts[1]
slide = prs.slides.add_slide(slide_layout)
title = slide.shapes.title
content = slide.placeholders[1]
title.text = "Introduction"
content.text = (
    "This presentation outlines the steps and considerations for establishing a human colony on Mars. "
    "It explores the challenges, necessary technology, and strategies for sustainable living on the Red Planet."
)

# Slide 3: Why Mars?
sli